# Packaging Projects for Distribution

- Creating a basic Python project with `setup.py` and `setup.cfg`
- Specifying dependencies
- Activating projects in a virtualenv with `setup.py develop`
- Distributing data with your project
- Using entry points to create console scripts
- Uploading source distributions to PyPI

# First, some terminology...

- a Python **module** is typically a single file ending in `.py` located somewhere along `sys.path` that you can use with the Python `import` statement
- a Python **package** is a folder located somewhere along `sys.path` containing a "magic" file `__init__.py` which can also be imported. If you import a package, Python is actually importing the `__init__.py` *module* in that *package*. You can also import modules or subpackages from a package.
- a Python **project** is a unit of distribution of Python code (it's something you can `pip install`)

# Creating a basic Python project with `setup.py` and `setup.cfg`

To create a project for distribution, you'll need to create a directory with:

- one or more Python packages to distribute
- a `setup.py` file
- (optionally) a `setup.cfg` file

In [ ]:
%%bash
rm -fr data/MyProject
mkdir -p data/MyProject/mypackage

In [ ]:
%%file data/MyProject/mypackage/__init__.py
print('This is the __init__ file for mypackage')

In [ ]:
%%file data/MyProject/mypackage/mymodule.py
print('This is mymodule')


def greet(name):
    print(f'Hello, {name}!')

In [ ]:
!find data/MyProject

For this demo, we'll use `setup.cfg` to provide metadata for our project, so we only need a minimal setup.py:

In [ ]:
%%file data/MyProject/setup.py
from setuptools import setup

setup(
#     name='MyProject',
#     version='0.1',
#     ...
)

We can create the `setup.cfg` file to specify how `setuptools` will build and distribute our project:

In [ ]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
version = 0.1
url = file:///
# author = Some Person
# author_email = somebody@example.com
# description = This should be a short description of our project
# long_description = file: README.md
# long_description_content_type = text/markdown
# classifiers =
#     Programming Language :: Python :: 3
#     Programming Language :: Python :: 3.7
#     Programming Language :: Python :: 3.8
# keywords = test, class

(Aside: you can use [cookiecutter](https://cookiecutter.readthedocs.io/en/1.7.2/) to get started on some projects)

It's always nice to provide a README as well:

In [ ]:
%%file data/MyProject/README.md
# MyProject

This project is a test setuptools project.

Its features:
    
* feature 1
* feature 2
* feature 3

In [ ]:
!find data/MyProject

## Creating a source distribution

The entry point for all our project management commands is `setup.py`.

We can create a simple source distribution of our project by calling `python setup.py sdist`:

In [ ]:
%%bash
cd data/MyProject
rm -fr dist
python setup.py sdist

In [ ]:
!ls data/MyProject/dist

In [ ]:
!pip install data/MyProject/dist/MyProject-0.1.tar.gz

In [ ]:
import mypackage

In [ ]:
!pip uninstall -y MyProject

In [ ]:
!tar tzf data/MyProject/dist/MyProject-0.1.tar.gz

## Adding our packages

So we have an empty project (no packages/modules). We need to tell setuptools to actually include our package explicitly:

In [ ]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
url = file:///
author = Some Person
author_email = somebody@example.com
version = 0.1
description = This should be a short description of our project
long_description = file: README.md
long_description_content_type = text/markdown

[options]
packages = mypackage

In [ ]:
%%bash
cd data/MyProject
python setup.py sdist

In [ ]:
!tar tzf data/MyProject/dist/MyProject-0.1.tar.gz

In [ ]:
!pip install data/MyProject/dist/MyProject-0.1.tar.gz

In [ ]:
import mypackage

In [ ]:
from mypackage import mymodule
mymodule.greet('Class')

In [ ]:
!pip uninstall -y MyProject

## Specifying dependencies

We can tell setuptools that we depend on particular versions (or version ranges) of other packages with an `install_requires` option:

In [ ]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
url = file:///
author = Some Person
author_email = somebody@example.com
version = 0.1
description = This should be a short description of our project
long_description = file: README.md
long_description_content_type = text/markdown

[options]
packages = mypackage
install_requires = 
    numpy>=1.16

In [ ]:
%%bash
cd data/MyProject
python setup.py sdist

In [ ]:
cat data/MyProject/MyProject.egg-info/requires.txt

# Activating projects using `setup.py develop`

When we're developing our project, we probably want its packages to be importable as though it were 'installed' in our virtualenv. To do this, we can invoke `setup.py` with the `develop` option. 

This creates a `MyProject.egg-link` file in a location along `sys.path` which makes your packages importable from anwhere that uses the virtualenv.

Note:

`pip install -e .` has equivalent effect to `python setup.py develop`

In [ ]:
%%bash
cd data/MyProject
rm -fr env
/usr/bin/python -m venv env
source env/bin/activate
# pip install -e .
python setup.py develop    # makes mypackage importable from anywhere in the venv

In [ ]:
cat data/MyProject/env/lib/python3.8/site-packages/easy-install.pth

In [ ]:
%%bash
source data/MyProject/env/bin/activate
cd /
python -c 'import mypackage.mymodule; mypackage.mymodule.greet("class")'

If we *don't* do `setup.py develop`, we *won't* be able to import the package:

In [ ]:
%%bash
source data/MyProject/env/bin/activate
pip uninstall -y MyProject   
cd /
python -c 'import mypackage.mymodule; mypackage.mymodule.greet("class")'

In [ ]:
%%bash
source data/MyProject/env/bin/activate
# cd data/MyProject; python setup.py develop 
pip install -e data/MyProject
cd /
python -c 'import mypackage.mymodule; mypackage.mymodule.greet("class")'

## Distributing data with our project

Normally, only Python files are included with our project. In order to include non-Python files, we need to specify those as well:

In [ ]:
%%file data/MyProject/mypackage/template.txt
This is an awesome template that greets you.

Hello, ${name}!

In [ ]:
%%file data/MyProject/mypackage/mymodule.py
import os, string


def greet(name):
    with open(os.path.join(
        os.path.dirname(__file__),
        'template.txt'
    )) as f:
        template = string.Template(f.read())
    print(template.safe_substitute({'name': name}))

In [ ]:
%%bash
cd data/MyProject
python setup.py sdist

In [ ]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
url = file:///
author = Some Person
author_email = somebody@example.com
version = 0.1
description = This should be a short description of our project
long_description = file: README.md
long_description_content_type = text/markdown


[options]
packages = mypackage
install_requires = 
    numpy
    
[options.package_data]
* = *.txt

In [ ]:
%%bash
cd data/MyProject
python setup.py sdist

You can also use include_package_data = true to include *all* data in package directories

In [ ]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
url = file:///
author = Some Person
author_email = somebody@example.com
version = 0.1
description = This should be a short description of our project
long_description = file: README.md
long_description_content_type = text/markdown


[options]
packages = mypackage
install_requires = 
    numpy
include_package_data = true
    

In [ ]:
%%bash
cd data/MyProject
python setup.py sdist

In [ ]:
%%bash
source data/MyProject/env/bin/activate
cd /
python -c 'import mypackage.mymodule; mypackage.mymodule.greet("class")'

# Use pkg_resources to access data files

In [ ]:
%%file data/MyProject/mypackage/mymodule.py
import string
import pkg_resources


def greet(name):
    filename = pkg_resources.resource_filename('mypackage', 'template.txt')
    with open(filename) as f:
        template = string.Template(f.read())
    print(template.safe_substitute({'name': name}))

In [ ]:
%%bash
source data/MyProject/env/bin/activate
cd /
python -c 'import mypackage.mymodule; mypackage.mymodule.greet("class")'

# Using entry_points for console_scripts

If you need to create a new command-line tool, a nice approach is to use the `entry_points` feature of `setuptools`:

In [ ]:
%%file data/MyProject/setup.cfg
[metadata]
name = MyProject
url = file:///
author = Some Person
author_email = somebody@example.com
version = 0.1
description = This should be a short description of our project
long_description = file: README.md
long_description_content_type = text/markdown


[options]
packages = mypackage
install_requires = 
    numpy>=1.16.0
    
[options.package_data]
* = *.txt

[options.entry_points]
console_scripts =
  my-greet=mypackage.mymodule:greet_main

In [ ]:
%%file data/MyProject/mypackage/mymodule.py
import sys
import string
import pkg_resources


def greet(name):
    filename = pkg_resources.resource_filename('mypackage', 'template.txt')
    with open(filename) as f:
        template = string.Template(f.read())
    print(template.safe_substitute({'name': name}))    
    
def greet_main():
    if len(sys.argv) > 1:
        name = ' '.join(sys.argv[1:])
    else:
        name = 'unknown human'
    greet(name)

In [ ]:
%%bash
cd data/MyProject
source env/bin/activate
python setup.py develop  # or pip install -e .

In [ ]:
!data/MyProject/env/bin/my-greet

In [ ]:
!data/MyProject/env/bin/my-greet Advanced Python

In [ ]:
cat data/MyProject/env/bin/my-greet

# Registering with PyPI

You'll need to create an account at http://pypi.org

In [ ]:
%%file data/MyProject/setup.cfg
[metadata]
;; change name to make it unique
name = ProductionalizingProject-1
url = https://github.com/DevelopIntelligence
author = Some Person
author_email = somebody@example.com
version = 0.16.0
description = This should be a short description of our project
long_description = file: README.md
long_description_content_type = text/markdown

[options]
packages = mypackage
install_requires = 
    numpy>=1.16.0
python_requires = >=3.6

[options.package_data]
* = *.txt

[options.entry_points]
console_scripts =
  my-greet=mypackage.mymodule:greet_main

In [ ]:
%%bash
cd data/MyProject
rm dist/*   # clean up old distributions
source env/bin/activate
pip install twine wheel
python setup.py sdist bdist_wheel
#twine upload dist/*

In [ ]:
!data/MyProject/env/bin/twine upload --help


In [ ]:
!data/MyProject/env/bin/twine  check data/MyProject/dist/*

In [ ]:
!/usr/bin/python -m venv --clear env-tmp

In [ ]:
%%bash
source env-tmp/bin/activate
pip install -U ProductionalizingProject-1

In [ ]:
%%bash
source env-tmp/bin/activate
pip freeze

In [ ]:
%%bash
source env-tmp/bin/activate
my-greet "Another Environment"

Clean things up

In [ ]:
!rm -r env-tmp data/MyProject

# Lab

Open [packaging lab][packaging-lab]

[packaging-lab]: ./packaging-lab.ipynb